In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Below code generates the vocabulary for english and telugu
# For telugu, I have used standard unicode points. Theses points should be changed if used other language 
# start token '<', end token: '>' and pad token: '?'
eng_characters = [chr(i) for i in range(ord('a'), ord('z')+1)]  # English lowercase letters
eng_characters += [chr(i) for i in range(ord('A'), ord('Z')+1)]  # English uppercase letters
eng_characters += [chr(i) for i in range(ord('0'), ord('9')+1)]  # Digits
eng_characters += ['?','<','>']

print(eng_characters)


# Define the range of Unicode code points for Telugu characters
TELUGU_START = 0x0C00
TELUGU_END = 0x0C7F

# Generate all Telugu characters
# start token '<', end token: '>' and pad token: '?'
telugu_characters = [chr(code_point) for code_point in range(TELUGU_START, TELUGU_END + 1)]
telugu_characters += ['?','<','>']

print(telugu_characters)



In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_train.csv'
valid_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_valid.csv'
test_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_test.csv'
# Load train and validation datasets

# Load the datasets
train_dataset = pd.read_csv(train_file, header=None)
valid_dataset = pd.read_csv(valid_file, header=None)
test_dataset = pd.read_csv(test_file, header=None)

# Extract the source and target columns
source_data = train_dataset.iloc[:, 0].values
target_data = train_dataset.iloc[:, 1].values


In [ ]:
# Add start and end tokens for each word in the dataset
source_data = ['<' + s + '>' for s in source_data]
target_data = ['<' + t + '>' for t in target_data]

In [ ]:
# Following code converts input letters to tensors (english and telugu)
loc = 1
e,t = train_dataset.iloc[loc]
char_to_index_eng = {char: i for i, char in enumerate(eng_characters)}
index_to_char_eng = {i: char for char, i in char_to_index_eng.items()}
def char_to_tensor_eng(word, char_to_index_eng):
    return torch.tensor([char_to_index_eng[char] for char in word], dtype=torch.long)

print(e)
print(char_to_tensor_eng(e,char_to_index_eng))

char_to_index_tel = {char: i for i, char in enumerate(telugu_characters)}
index_to_char_tel = {i: char for char, i in char_to_index_tel.items()}
def char_to_tensor_tel(word, char_to_index_tel):
    return torch.tensor([char_to_index_tel[char] for char in word], dtype=torch.long)

print(t)
t1 = char_to_tensor_tel(t,char_to_index_tel)
print(t1)

characters = [telugu_characters[char_int.item()] for char_int in t1]
string = ''.join(characters)
# Print the characters
print("Characters:", string)

In [ ]:
# Iterate for each word in the dataset
source = [char_to_tensor_eng(s, char_to_index_eng) for s in source_data]
target = [char_to_tensor_tel(s, char_to_index_tel) for s in target_data]

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
# Pad the tensors to the same length
source_padded = pad_sequence(source, batch_first=True, padding_value=62)
target_padded = pad_sequence(target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(source_padded.size(1), target_padded.size(1))

# Pad source and target to the same length
source_padded = torch.nn.functional.pad(source_padded, (0, max_length - source_padded.size(1)), value=62)
target_padded = torch.nn.functional.pad(target_padded, (0, max_length - target_padded.size(1)), value=128)


In [ ]:
# Below function converts a given tensor of source, target to english, telugu characters
def t2p(s,t):
    characters = [eng_characters[char_int.item()] for char_int in s]
    string1 = ''.join(characters)
#     print(string)
    characters = [telugu_characters[char_int.item()] for char_int in t]
    string2 = ''.join(characters)
    print(f"{string1} {len(string1)} : {string2} {len(string2)} :")
t2p(source_padded[0],target_padded[0])

In [ ]:
# Create a TensorDataset
dataset = TensorDataset(source_padded, target_padded)

# Create a DataLoader
# batch_size = 64
def train_loader(batch_size):
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return train_dataloader



In [ ]:
#valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
val_source_data = valid_dataset.iloc[:, 0].values
val_target_data = valid_dataset.iloc[:, 1].values

val_source_data = ['<' + s + '>' for s in val_source_data]
val_target_data = ['<' + t + '>' for t in val_target_data]

val_source = [char_to_tensor_eng(s, char_to_index_eng) for s in val_source_data]
val_target = [char_to_tensor_tel(s, char_to_index_tel) for s in val_target_data]

val_source_padded = pad_sequence(val_source, batch_first=True, padding_value=62)
val_target_padded = pad_sequence(val_target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(val_source_padded.size(1), val_target_padded.size(1))

# Pad source and target to the same length
val_source_padded = torch.nn.functional.pad(val_source_padded, (0, max_length - val_source_padded.size(1)), value=62)
val_target_padded = torch.nn.functional.pad(val_target_padded, (0, max_length - val_target_padded.size(1)), value=128)
# Create a TensorDataset
val_dataset = TensorDataset(val_source_padded, val_target_padded)

def val_loader(batch_size):
    valid_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    return valid_dataloader

In [ ]:
# Below is the class definitions for Encoder, Decoder, seq2seq module
# Encoder consists of embedding layer, hidden layers
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size, dropout_in):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.cell_type = cell_type
        self.embeddin_size = embedding_size
        self.dropout_in = dropout_in
        # Character embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout_in,  batch_first=True)
        
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        # embedded shape: (seq_length:max_length, N(batch size:32), embedding_size:100)
        output, hidden = self.rnn(embedded, hidden)
        return hidden
    
    def getInitialState(self):
        return torch.zeros(self.num_layers,self.batch_size,self.hidden_size).to(device)

# Decoder consists of embedding layer, hidden layers and output layer
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type, dropout_in):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.embeddin_size = embedding_size
        self.dropout_in = dropout_in
        
        # Devanagari character embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_in, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout_in,  batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)  
        predictions = self.softmax(self.fc(output))       
        return predictions, hidden

# Following is the class definition for seq2seq module
# This class uses encoder and decoder module and establishes connection between them
# It uses default teacher forcing ratio of 0.5. In case of inference, this must be 0 (provision for parameter pass)
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_letter_size = 131
        
        outputs = torch.zeros(target_len, batch_size, target_letter_size).to(device)
        
        # Encoder forward pass
        if(self.cell_type == "LSTM"):
            hidden = (self.encoder.getInitialState(), self.encoder.getInitialState())
        else:
            hidden = self.encoder.getInitialState()
        hidden = self.encoder(source, hidden)    
        x = target[:,0].view(batch_size, 1)

        # sequentially call the decoder module
        for t in range(0, target_len):
            output, hidden = self.decoder(x, hidden)
            output = output[:, -1, :]
            outputs[t] = output
            best_guess = output.argmax(1)
            best_guess = best_guess.view(batch_size, 1) 
            x = target[:,t].view(batch_size,1) if np.random.rand() < teacher_forcing_ratio else best_guess
        
        return outputs


In [ ]:
# Define the model architecture
input_size = len(char_to_index_eng)
output_size = len(char_to_index_tel)
# hidden_size = 256
# embedding_size = 256
# num_layers = 1
# cell_type = 'LSTM'
# LEARNING_RATE = 0.0001
# NUM_EPOCHS = 10

# Following is the train function, that takes the sweep configuration and train the seq2seq model using BP

def train(hidden_size, embedding_size, 
          num_layers, cell_type, LEARNING_RATE, NUM_EPOCHS, batch_size, optimizer_in, dropout_in):

    train_dataloader = train_loader(batch_size)
    valid_dataloader = val_loader(batch_size)
    encoder_net = Encoder(input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size, dropout_in).to(device)
    decoder_net = Decoder(output_size, embedding_size, hidden_size, 
                     output_size, num_layers, cell_type, dropout_in).to(device)
    model = Seq2Seq(encoder_net, decoder_net, cell_type).to(device)

    criterion = nn.NLLLoss()
    if(optimizer_in == 'adam'):
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    else:
        optimizer = optim.NAdam(model.parameters(), lr=LEARNING_RATE)

    step = 0
    # Train the model
    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        total_correct = 0
        accuracy = 0
        for batchid,(inp_data,target) in enumerate(tqdm(train_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target)
            indices = output.argmax(2)

            if(batchid==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])

            optimizer.zero_grad()
            loss = 0
            for i in range(output.shape[0]):
                loss += criterion(output[i], target[:,i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            total_loss += loss.item()/ len(inp_data[0])
            #accuracy += total_correct/batch_size
            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
            optimizer.step()
            step+=1

        train_loss = total_loss / len(train_dataloader)
        train_accuracy = total_correct/ len(train_dataloader.dataset)
        validation_loss, validation_accuracy = calculate_validation_loss(model, criterion, valid_dataloader, batch_size)
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")
        print("train_acc", train_accuracy)
        print("train_loss", train_loss)
        print("valid_acc", validation_accuracy)
        print("valid_loss", validation_loss)
        
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})


In [ ]:
# Following is the validation loss/accuracy calculation function
# teacher forcing must be zero
def calculate_validation_loss(model, criterion, valid_dataloader,batch_size):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    accuracy = 0
    total_correct = 0
    step = 0
    with torch.no_grad():
        for batchid,(inp_data,target) in enumerate(tqdm(valid_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target, 0) # teacher - forcing to 0
            indices = output.argmax(2)
            output_dim = output.shape[-1]
            if(batchid==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])
                print((indices[:,0] == target[0]).sum().item())

            # Flatten the output and target tensors to compute the loss
            val_loss = 0
            
            for i in range(output.shape[0]):
                val_loss += criterion(output[i], target[:, i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            step +=1
            total_val_loss += val_loss.item() / len(inp_data[0])
            #accuracy += total_correct/batch_size

    avg_val_loss = total_val_loss / len(valid_dataloader)
    accuracy = total_correct/len(valid_dataloader.dataset) #/ len(valid_dataloader)
    return avg_val_loss, accuracy


In [ ]:
# train(hidden_size, embedding_size, 
#           num_layers, cell_type, LEARNING_RATE, NUM_EPOCHS)

In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
# def main_fun():
#     wandb.init(project ='Assignment3')
#     params = wandb.config
#     with wandb.init(project = 'Assignment3', name='epochs_'+str(params.num_epochs) 
#                     +'layers_'+str(params.num_layers) + 'embedding_'+str(params.embedding_size)
#                     +'hidden_size_'+str(params.hidden_size) + 'cell_type_'+str(params.cell_type)
#                     +'lear_rate_'+str(params.learning_rate) + 'batch_size'+str(params.batch_size)
#                     + 'optim_'+str(params.optimizer) + 'dropout_'+str(params.dropout_in)) as run:
#         train(params.hidden_size,params.embedding_size,
#               params.num_layers, params.cell_type, params.learning_rate, 
#               params.num_epochs, params.batch_size,params.optimizer, params.dropout_in)

# sweep_params = {
#     'method' : 'bayes',
#     'name'   : 'cs23m005',
#     'metric' : {
#         'goal' : 'maximize',
#         'name' : 'val_accuracy',
#     },
#     'parameters' : {
#             'num_epochs':{'values':[20,25,30]},
#             'learning_rate' :{'values':[1e-4, 1e-3]},
#             'num_layers' :{'values':[1,2,3,4,5]},
#             'embedding_size' :{'values':[128,256]},
#             'hidden_size' :{'values':[128,256]},
#             'cell_type' :{'values':['LSTM', 'GRU', 'RNN']},
#             'batch_size' :{'values':[64,32,128]},
#             'optimizer' :{'values':['adam','nadam']},
#             'dropout_in' :{'values':[0.2,0.3,0.5]}
            
#     }
# }
# sweepId = wandb.sweep(sweep_params,project = 'Assignment3')
# wandb.agent(sweepId,function =main_fun,count = 1)
# wandb.finish()

In [ ]:
import argparse

def parse_args():
    p = argparse.ArgumentParser(description = "provide optinal parameters for training")
    p.add_argument('-wp', '--wandb_project', type=str, default="Assignment3", help="wandb project name")
    p.add_argument('-opt', '--optimizer', type=str, default="nadam", choices = ['adam','nadam'], help="optimizer for backprop")
    p.add_argument('-bS', '--batch_size', type=int, default=32, choices = [32, 64, 128, 256], help="batch size")
    p.add_argument('-nE', '--num_epochs', type=int, default=25, choices = [5, 10], help="number of epochs")
    p.add_argument('-lR', '--learning_rate', type=float, default=1e-3, choices = [1e-3, 1e-4], help="learning rate")
    p.add_argument('-hS', '--hidden_size', type=int, default=128, choices = [32, 64, 128, 256], help="hidden size")
    p.add_argument('-eS', '--embedding_size', type=int, default=256, choices = [32, 64, 128, 256], help="embedding size")
    p.add_argument('-nL', '--num_layers', type=int, default=3, choices = [1,2,3,4,5], help="number of layers")
    p.add_argument('-cell', '--cell_type', type=str, default="LSTM", choices = ['LSTM', 'GRU', 'RNN'], help="cell type")
    p.add_argument('-dr', '--dropout_in', type=float, default=0.2, choices = [0,0.2,0.3,0.5], help="drop out")
    

args = parse_args()
wandb.init(project = args.wadb_project)
wandb.run.name=f'optimizer {str(args.optimizer)} epochs {str(args.num_epochs)} learning rate {args.learning_rate}'

train(args.hidden_size,args.embedding_size,
              args.num_layers, args.cell_type, args.learning_rate, 
              args.num_epochs, args.batch_size,args.optimizer, args.dropout_in)